In [2]:
# Import all necessary packages
import os
from os import sep
import pubchempy as pcp
import pandas as pd
from collections import defaultdict
import numpy as np
from numpy import nan
from IPython.display import Image
import tabula

from tabula import read_pdf
from tabulate import tabulate

In [3]:
import geckodriver_autoinstaller
geckodriver_autoinstaller.install();

from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

import csv
import requests
import time


## Import Data

In [4]:
# Read in list of chemicals data file and change the name of column.
dataset = pd.read_csv ("/Users/celinyounan/Desktop/swiss_adme/data/list_of_chemicals.csv")
dataset.rename(columns = {'Chemical name' : 'Chemical_Name'}, inplace = True)
# (UNFOUND)imprort list of chemical data for chemicals that where orinionally not found by pubchem and chance name of column
dataset_unfound = pd.read_csv ("/Users/celinyounan/Desktop/swiss_adme/data/list_of_chemicals_unfound.csv")
dataset_unfound.rename(columns = {'Chemical name' : 'Chemical_Name'}, inplace = True)


In [5]:
# Extract column ad new dataset for analysis.
# For origional list 
ChemList_Column = dataset["Chemical_Name"]
# For unfound
ChemList_Column_unfound = dataset_unfound["Chemical_Name"]

## PubChem
### Using pubchempy package

In [6]:
# Use PubChempy to obtain chemical identifiers and insert into cidlist created for exact number of chemicals.
cidlist = [] # list of cids
unfoundcid = [] # list of unfound chemicals
foundcid = []  # list of found chemicals
for i,d in enumerate(ChemList_Column):
    values = pcp.get_cids(d)
    if len(values) > 0:
        cidlist.append(values[0])
        foundcid.append(d)
    else:
        unfoundcid.append(d)
        
# for unfound 
# Use PubChempy to obtain chemical identifiers and insert into cidlist created for exact number of chemicals.
cidlist_unfound = [] # list of cids
unfoundcid_unfound = [] # list of unfound chemicals
foundcid_unfound = []  # list of found chemicals
for i,d in enumerate(ChemList_Column_unfound):
    values = pcp.get_cids(d)
    if len(values) > 0:
        cidlist_unfound.append(values[0])
        foundcid_unfound.append(d)
    else:
        unfoundcid_unfound.append(d)

In [7]:
# cidlist and foundcid size
(len(cidlist),len(foundcid))

(157, 157)

In [8]:
# for unfound
# cidlist and foundcid size
(len(cidlist_unfound),len(foundcid_unfound))

(14, 14)

In [9]:
# Use identifiers to obtain properties of the chemicals (MolecularFormula, MW, and SMILES) in place in correct list
# Runs in 4 minutes
count = len(cidlist)
Formula, Weight, Smiles = [None] * count,[None] * count,[None] * count

for i,d in enumerate(cidlist):
    if d != None:
        Formula[i] = pcp.get_properties(['MolecularFormula'],d)[0]['MolecularFormula']
        Weight[i] = float(pcp.get_properties(['MolecularWeight'], d)[0]['MolecularWeight'])
        Smiles[i] = pcp.get_properties(['CanonicalSMILES'],d)[0]['CanonicalSMILES']
        
#for unfound 
# Use identifiers to obtain properties of the chemicals (MolecularFormula, MW, and SMILES) in place in correct list
# Runs in 4 minutes
count_unfound = len(cidlist_unfound)
Formula_unfound, Weight_unfound, Smiles_unfound = [None] * count_unfound,[None] * count_unfound,[None] * count_unfound

for i,d in enumerate(cidlist_unfound):
    if d != None:
        Formula_unfound[i] = pcp.get_properties(['MolecularFormula'],d)[0]['MolecularFormula']
        Weight_unfound[i] = float(pcp.get_properties(['MolecularWeight'], d)[0]['MolecularWeight'])
        Smiles_unfound[i] = pcp.get_properties(['CanonicalSMILES'],d)[0]['CanonicalSMILES']

## SwissADME
---

In [22]:
# SwissADME code
# THIS SECTION TAKES ~20 MIN

# Create list for SwissADME links
SwissAdme_links = [None] * len(Smiles)

# Name each elements for code
name = "smiles"
button = "submitButton"
path = '//*[@id="sib_body"]/div[7]/a[1]'
website = "http://www.swissadme.ch"
footer = '//*[@id="sib_footer"]'


# This will run headless - runs in the background
options = Options()
options.headless = True
driver = Firefox(options=options)
wait = WebDriverWait(driver,30)
# Open SwissADME database
driver.get(website)
    
# Locate search box on SwissADME and import the smiles from previously created list
element1 = wait.until(EC.visibility_of_element_located((By.NAME, name)))
smiles_search_ADME = driver.find_element(By.NAME,name)
# Loop through and input all smiles in list
for i,d in enumerate(Smiles):
    smiles_search_ADME.send_keys(d)
    smiles_search_ADME.send_keys(Keys.RETURN)

# Loop through and input all smiles in list
for i,d in enumerate(Smiles_unfound):
    smiles_search_ADME.send_keys(d)
    smiles_search_ADME.send_keys(Keys.RETURN)

# Find and click Submit
element2 = wait.until(EC.presence_of_element_located((By.ID, button)))
run = driver.find_element(By.ID,button).submit()

element3 = wait.until(EC.presence_of_element_located((By.XPATH, footer)))
    # Locate href link (csv link)
element4 = wait.until(EC.presence_of_element_located((By.ID,"mol-cell-1")))
csvlinks_ADME = driver.find_element(By.XPATH, path).get_property('href')

SwissAdme_links[i] = csvlinks_ADME
# this allows time for the data to load
time.sleep(1500)
# Close browser
# driver.close()   

In [23]:
# create emply list the length of swissadme_links
SwissAdme_data = [None] * len(SwissAdme_links)
# download csv from list
SwissAdme_data =  pd.read_csv(csvlinks_ADME)

In [24]:
# view downloaded data
SwissAdme_data

,Molecule,Canonical SMILES,Formula,MW,#Heavy atoms,#Aromatic heavy atoms,Fraction Csp3,#Rotatable bonds,#H-bond acceptors,#H-bond donors,...,Lipinski #violations,Ghose #violations,Veber #violations,Egan #violations,Muegge #violations,Bioavailability Score,PAINS #alerts,Brenk #alerts,Leadlikeness #violations,Synthetic Accessibility
0,Molecule 1,OCC1OC(OC(c2ccccc2)C#N)C(C(C1O)O)O,C14H17NO6,295.29,21,6,0.50,4,7,4,...,0,1,0,0,0,0.55,0,0,0,4.21
1,Molecule 2,OCC1OC(OCC2OC(OC(c3ccccc3)C#N)C(C(C2O)O)O)C(C(...,C20H27NO11,457.43,32,6,0.65,7,12,7,...,2,1,1,1,4,0.17,0,0,1,5.41
2,Molecule 3,Cc1cc(O)c2c(c1)C(=O)c1c(C2=O)c(O)cc(c1)O,C15H10O5,270.24,20,12,0.07,0,5,3,...,0,0,0,0,0,0.55,1,0,0,2.57
3,Molecule 4,CN(CCc1ccc(cc1)O)C,C10H15NO,165.23,12,6,0.40,3,2,1,...,0,0,0,0,1,0.55,0,0,1,1.00
4,Molecule 5,CNCCc1ccc(cc1)OC,C10H15NO,165.23,12,6,0.40,4,2,1,...,0,0,0,0,1,0.55,0,0,1,1.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Molecule 167,COc1cc2ccc(=O)oc2cc1O,C10H8O4,192.17,14,10,0.10,1,4,1,...,0,0,0,0,1,0.55,0,1,1,2.62
167,Molecule 168,COc1ccc(cc1)c1cc(=O)c2c(o1)cc(cc2O)O,C16H12O5,284.26,21,16,0.06,2,5,2,...,0,0,0,0,0,0.55,0,0,0,2.98
168,Molecule 169,OCC1OC(Oc2cc3oc(=O)ccc3cc2O)C(C(C1O)O)O,C15H16O9,340.28,24,10,0.40,3,9,5,...,0,1,1,1,0,0.55,0,1,0,4.75
169,Molecule 170,COc1c(O)c(OC)cc2c1oc(=O)cc2,C11H10O5,222.19,16,10,0.18,2,5,1,...,0,0,0,0,0,0.55,0,1,1,3.02


In [25]:
# Insert Chemical Names to results
SwissAdme_data.insert(0, 'Chemical_Name', foundcid)

ValueError: Length of values (157) does not match length of index (171)

In [ ]:
# Drop useless column
SwissAdme_data.drop(['Molecule'], axis=1,inplace=True)

In [ ]:
# view final data
SwissAdme_data

,Chemical_Name,Canonical SMILES,Formula,MW,#Heavy atoms,#Aromatic heavy atoms,Fraction Csp3,#Rotatable bonds,#H-bond acceptors,#H-bond donors,...,Lipinski #violations,Ghose #violations,Veber #violations,Egan #violations,Muegge #violations,Bioavailability Score,PAINS #alerts,Brenk #alerts,Leadlikeness #violations,Synthetic Accessibility
0,Artemisia ketone,C=CC(C(=O)C=C(C)C)(C)C,C10H16O,152.23,11,0,0.50,3,1,0,...,0,1,0,0,2,0.55,0,2,1,2.67
1,Artemisyl acetate,C=CC(C(C=C(C)C)OC(=O)C)(C)C,C12H20O2,196.29,14,0,0.58,5,2,0,...,0,0,0,0,1,0.55,0,1,2,3.23
2,Beta-ocimene,C=CC(=CCC=C(C)C)C,C10H16,136.23,10,0,0.40,3,0,0,...,0,1,0,0,2,0.55,0,2,2,3.63
3,Beta-pinene,C=C1CCC2CC1C2(C)C,C10H16,136.23,10,0,0.80,0,0,0,...,1,1,0,0,2,0.55,0,1,2,3.73
4,Beta-thujone,O=C1CC2(C(C1C)C2)C(C)C,C10H16O,152.23,11,0,0.90,1,1,0,...,0,1,0,0,2,0.55,0,0,1,2.79


In [ ]:
# Export ADME as csv. 
SwissAdme_data.to_csv('../Output/FOUND_ADME_data_test.csv', index=False)